In [ ]:
# Import the Earth Engine Python API
import ee
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import geemap

# Initialize the API
ee.Initialize()

In [ ]:
area_nova = geemap.geojson_to_ee("/home/josue/Documentos/sentinel/ndvi_lumet/area_nova.geojson")

In [ ]:
image = "COPERNICUS/S2_SR_HARMONIZED"

# Load the LANDSAT data
collection = ee.ImageCollection(image)

# Select the band of interest
collection = collection.select(['B4', 'B8']).filterDate('2015-01-01', '2023-01-01').filterBounds(area_nova)

# Define a function to extract the value and date for each image
def extract(image):
    reduced = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=area_nova, maxPixels=1e9)
    nir = reduced.get('B8')
    red = reduced.get('B4')
    date = image.date().format('YYYY-MM-dd')
    return ee.Feature(None, {"red": red, "nir": nir, 'date': date})

# Map the function over the collection and get the results
results = collection.map(extract).getInfo()

In [ ]:
df = pd.DataFrame.from_records([f['properties'] for f in results['features']])
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

df["NDVI"] = (df["nir"] - df["red"]) / (df["nir"] + df["red"])

df["NDVI"].plot()

# # qa pixel filtering
# # https://hyspeedblog.wordpress.com/2014/08/27/working-with-landsat-8-using-and-interpreting-the-quality-assessment-qa-band/
# df["QA_PIXEL"] = df["QA_PIXEL"].apply(extract_qa_info)
# df["CLOUD"] = df["QA_PIXEL"].apply(bin_condition, slice=slice(0,2))
# df["CIRRUS"] = df["QA_PIXEL"].apply(bin_condition, slice=slice(2,4))
# df["SNOW"] = df["QA_PIXEL"].apply(bin_condition, slice=slice(4,6))
# df["VEGETATION"] = df["QA_PIXEL"].apply(bin_condition, slice=slice(6,8))
# df["SHADOW"] = df["QA_PIXEL"].apply(bin_condition, slice=slice(8,10))

# # Remove pixels with cloud and cirrus
# df.loc[(df["CLOUD"]<2) & (df["CIRRUS"]<2)].dropna(subset=["CLOUD", "CIRRUS"])["NDVI"].loc["2015":"2018"].plot()